Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [40]:
NAME = "Lars Janssen"

---

For those not familiar with Python, a quick overview is given [here](https://github.com/palcu/python-for-competitive-programming/blob/master/python-for-competitive-programming.ipynb).

# Notebook week 3: Graphs and Depth-First Search

In [41]:
# Run the following piece of code!

# This asserts that we are running Python 3.
import sys
assert sys.version_info >= (3,)

# The following code allows us to emulate command line I/O.
from io import StringIO
from sys import stdin
# Overwrite the jupyter input function.
def input():
    return stdin.readline()

## Exercise 1: Input processing
#### Edge lists and adjacency lists
The very first step in solving any graph problem is getting the data into a useful structure. In coding competitions, graphs are usually given in the *edge list* format, but the *adjacency list* is often a much more efficient data structure. An edge list is just a list of all edges in the graph. An adjacency list consists of a list of lists: one list for every node, which contains the neighbors of that node.

#### 0-based and 1-based graphs
When you have a graph with $N$ nodes, you can choose to number the vertices as $1, 2, ..., N$ or as $0, 1, ..., N-1$. The edge lists you get as input can be either 0-based or 1-based. (In the newer APC/BAPC contests, they will usually be 1-based.) For your program internally, keeping track of it in a 0-based way is almost always more convenient.

Therefore, the first step in most graph-based algorithms is converting the input from either a 0-based edge list or a 1-based edge list to a 0-based adjacency list.

#### Undirected and directed graphs
Although reading directed and undirected graphs is similar, it does not work quite the same. Usually when working with directed graphs, you want to keep _two_ adjacency lists: one for "outgoing edges" and one for "incoming edges". For a standard depth-first-search on a directed graph, you will only need outgoing edges though.

In this notebook we work only with undirected graphs, but there may be some exercises with directed graphs in the Kattis competition!

#### Exercise 
Finish the following function to convert from *edge list* to *adjacency list*.

In [42]:
def edge2adj_undirected(V, E, edges):
    """ Given an undirected graph as a 1-based edge list,
    produce a 0-based adjacency list. """
    assert len(edges) == E
    adjlist = [[] for _ in range(V)]
    for (v, w) in edges:
        assert 1 <= v <= V and 1 <= w <= V
        v = v - 1
        w = w - 1
        adjlist[v].append(w)
        adjlist[w].append(v)
    return adjlist

In [43]:
# Test that the function returns the example from the slides.
# If it raises an IndexError, you are most likely off-by-one.
edges = [(1,2), (1,5), (2,3), (2,5), (3,4), (4,5), (4,6)]
try:
    result = edge2adj_undirected(V=6, E=7, edges=edges)
except IndexError:
    assert False, "is it possible you are off-by-one?"

assert result != [[2,5], [1,3,5], [2,4], [3,5,6], [1,2,4], [4]], "You made a 1-based adjacency list, not 0-based."
assert result != [[1,4], [2,4], [3], [4,5], [], []], "If u is connected to v, don't forget to connect v to u: " \
    "we are working with a directed graph."
assert result == [[1,4], [0,2,4], [1,3], [2,4,5], [0,1,3], [3]]

## Exercise 2: Adjacency list application
Use the cell below to find the number of "neighbours-of-neighbours", i.e. the number of different vertices that you can find by walking two edges from a starting vertex.

In [44]:
def nbrs_of_nbrs(adjlist):
    """Given a 0-based adjacency list as input, give
    a list with the number of neighbors-of-neighbors of each node."""
    number = [0 for i in range (len(adjlist))]
    for i in range(len(adjlist)):
        nbrsofnbrs = []
        nbrs = adjlist[i]
        for y in nbrs:
            for p in adjlist[y]:
                if(p not in nbrsofnbrs):
                    number[i] += 1
                    nbrsofnbrs.append(p)
    return number

In [45]:
adjlist = [[1, 4], [0, 2, 4], [1, 3], [2, 4, 5], [0, 1, 3], [3]]
assert nbrs_of_nbrs(adjlist) != [6, 7, 6, 6, 8, 3], "Did you maybe count some neighbour-neighbours twice?"
assert nbrs_of_nbrs(adjlist) == [5, 4, 4, 3, 5, 3]

## Exercise 3: Recursive DFS example
DFS can be used to determine which vertices can be seen from a given starting vertex. Use the slides to finish the code below.

In [54]:
def DFS(v, adjlist, seen):
    """
    Use the DFS algorithm to fill the list `seen`
    (initially all False) with `True`s whenever a node is
    reachable from the initial node `v`.
    """
    seen[v] = True
    for nbr in adjlist[v]:
        if not seen[nbr]:
            DFS(nbr, adjlist, seen)

In [55]:
# Test that for this connected graph, DFS'ing from *any* node sees every node.
adjlist = [[1, 4], [0, 2, 4], [1, 3], [2, 4, 5], [0, 1, 3], [3]]
for v in range(len(adjlist)):
    seen = [False for _ in adjlist]
    try:
        DFS(v, adjlist, seen)
        assert all(seen)
    except IndexError:
        assert False, "Are you off-by-one?"
    
# Test that for this unconnected graph, there are unseen nodes.
adjlist = [[1,2], [0], [0], [4], [3]]
seen = [False for _ in adjlist]
DFS(1, adjlist, seen)
assert seen == [True, True, True, False, False]

## Interlude 3.5: recursion limits

In many languages (Python being an example) doing very deep recursion is not a good idea. Therefore, Python has a builtin recursion limit, and will give an exception if you try to do any deeper recursion.

In [56]:
n = sys.getrecursionlimit()
print(f"The recursion limit is: {n}")
print(f"Let's try to do DFS on a graph that's basically a long line with {n} nodes.")

long_graph = [[1]] + [[k-1, k+1] for k in range(1, n+5)] + [[n+4]]
DFS(0, long_graph, [False for _ in range(n+6)])

The recursion limit is: 3000
Let's try to do DFS on a graph that's basically a long line with 3000 nodes.


RecursionError: maximum recursion depth exceeded

You can change the recursion limit of Python with the command `sys.setrecursionlimit(N)`. But if you do, you quickly find out why the limit exists: if you try to recur too deeply, Python will just die on you. (If you recur too deeply in a notebook, your browser might crash.) How deep you can go depends on the configuration of your computer and Python installation. On Mees' laptop, it's about 16000. On the Kattis computers, it's about 5000.

None of this is dangerous, but it means we cannot always use a recursion-based DFS even if it would solve the problem. In particular, the Kattis problem from the next exercise cannot be solved (on Kattis) recursively. Next week we will learn a way around this, and see how we can do a DFS without recursion. 

The Kattis problems from this week's set of problems _should_ all be solvable on Kattis with a recursive DFS; but you might have to up the limit a bit with `sys.setrecursionlimit(N)` at the start of your program.

## Exercise 4: Where's My Internet??
We will solve the Kattis problem [Where's My Internet??](https://open.kattis.com/problems/wheresmyinternet). Read the problem statement, and finish the code below.

In [73]:
def wheres_my_internet():
    # Read first line: V vertices, E edges
    V, E = [int(x) for x in input().split()]

    # Create an empty edge list and read edges into it.
    edges = []
    for _ in range(E):
        a, b = [int(x) for x in input().split()]
        edges.append([a,b])
    
    # Convert the 1-based edge list you read to a 0-based adjacency list.
    adjlist = edge2adj_undirected(V, E, edges)
    for v in range(len(adjlist)):
        seen = [False for _ in adjlist]
    DFS(1, adjlist, seen)

    for i in range(len(seen)):
        if(seen[i] == False):
            print(i + 1)
    # Solve the problem using the previous exercises. Print the result.
    

In [74]:
# TEST that the function prints the right solution
stdin = StringIO('6 4\n1 2\n2 3\n3 4\n5 6\n')
wheres_my_internet()
# Verify yourself that this prints:
# 5
# 6

# If it prints a 4 and a 5 instead, don't forget: the question
# works with a 1-indexed graph, and you are working
# with a 0-indexed graph!


5
6
